In [14]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import yfinance as yf
from machine_learning_finance import analyze_trades, \
    calc_probabilties_without_lookahead, plot_backtest_analysis, \
    metrics_to_dataframe, create_train_test_windows, make_inverse_env_for
from plotly.offline import init_notebook_mode

init_notebook_mode(connected=True)
windows = [300, 600, 900, 1500]

symbol = "QQQ"
inverse = "PSQ"
file = f"../backtests/backtest_inverse_{symbol}_365_h0.9_l0.01.csv"
period = 365
pd.options.display.max_rows = None


def plot_win_loss(file, metrics):
    ledger = pd.read_csv(file)
    # Set the style for the plots
    sns.set(style='whitegrid')

    # Create a bar plot for profit and loss stats
    profit_stats, loss_stats = metrics['profit_stats'], metrics['loss_stats']
    profit_loss_df = pd.DataFrame([profit_stats, loss_stats], columns=['min', 'max', 'mean', 'median', 'std'], index=['profit', 'loss'])

    plt.figure(figsize=(12, 6))
    ax = sns.barplot(data=profit_loss_df.transpose(), palette='muted')
    ax.set_title('Profit and Loss Statistics')
    ax.set_ylabel('Value')

    # Show the plot
    plt.show()

def analyze_and_graph_range(file, symbol, start, end, df):
    ledger = pd.read_csv(file)
    hist_df, test_df = create_train_test_windows(df, start=start, end=end)    
    test_df = calc_probabilties_without_lookahead(test_df, hist_df, hist_cutoff="2023-05-15")
    plot_backtest_analysis(test_df, ledger, inverse=symbol)
    return ledger, test_df
   
def analyze_and_graph(file, symbol, period, df):
    ledger = pd.read_csv(file)
    hist_df, test_df = create_train_test_windows(df)  
    df = df.tail(period)
    df = calc_probabilties_without_lookahead(df, hist_df)
    plot_backtest_analysis(df, ledger, inverse=symbol)

    return ledger, df

ticker_obj = yf.download(tickers=symbol)
df = pd.DataFrame(ticker_obj)
#plot_win_loss(file, metrics)

#print(ledger)

#print(metrics_df)
#df.sort_index(ascending=False)

[*********************100%***********************]  1 of 1 completed


In [15]:
ledger, df2 = analyze_and_graph(file, symbol, 365*4, df)    
#ledger, df2 = analyze_and_graph_range(file, symbol, "03-27-2020", "12-17-2021", df)

# Display metrics as text
metrics = analyze_trades(ledger, symbol, period)
metrics["file"] = file
metrics_df = metrics_to_dataframe(metrics) 

merged_df = ledger.merge(df2, left_on='Date', right_index=True, how='left')
merged_df[["Date", "Price", "Product", "Side", "Action","prob_above_trend", "Value"]]

[*********************100%***********************]  1 of 1 completed


,Date,Price,Product,Side,Action,prob_above_trend,Value
0,2022-01-11,11.241300,PSQ,long,enter,0.448959,4938.194099
1,2022-03-14,13.392600,PSQ,long,exit,0.998049,5925.693646
2,2022-03-14,321.351714,QQQ,long,enter,0.998049,5853.608810
3,2023-02-02,314.837201,QQQ,long,exit,0.008451,5779.450516
4,2023-02-02,12.715900,PSQ,long,enter,0.008451,5707.969189
5,2023-06-23,10.857500,PSQ,long,exit,0.007305,4910.850501


In [16]:
env = make_inverse_env_for(symbol,
                        inverse,
                        1,
                        period,
                        prob_high=0.9,
                        prob_low=0.1)
print("env version:", env.env_version)
env.expert_opinion_df()
env.timeseries[["prob_above_trend", "action"]]

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
env version: 2


,prob_above_trend,action
Date,,
2022-01-10,0.015683,2
2022-01-11,0.006171,2
2022-01-12,0.004644,2
2022-01-13,0.022352,2
2022-01-14,0.015919,2
2022-01-18,0.055478,2
2022-01-19,0.086979,2
2022-01-20,0.138618,0
2022-01-21,0.304036,0


In [ ]:
merged_df[["Date", "Price", "Product", "Side", "Action","Value"]]